Ideas for what to do next: 
- do Gitty stuff like repos and cloning and pushing

- ~Try to get this working in Jupyter~

- ~Get the policies~

- Get all policy details related to climate?

- Loop through all division details to try to get & store them all

- Try to get just the divisions associated with the climate policies

- Try some python dataviz,  

- Get PowerBI licence and do some dataviz

- Get tweets and link them to the MPs

- Try to get data into redshift or mongodb

- Should I be storing the api key as a variable? I don't think so, should be fine to just have written out each time right?

# First some setup

In [1]:
#Import libraries
import requests
import json
import pandas as pd
import os

#pd.show_versions()
#pd.__version__

In [9]:
os.getcwd()

'C:\\Users\\Oona\\Data Projects\\Climate Change\\Scripts'

# Now look at people

In [18]:
##get the list of people in Parliament into a dataframe

#Api call
mps_get = requests.get("https://theyvoteforyou.org.au/api/v1/people.json?key=HsjbG%2FoZdGMFMbN4PK%2F1")

#Print the api status to confirm the connection etc
print("api status: ", mps_get.status_code)

#pull the content from the api
mps_content = mps_get.content

#Load the content into a Python object
loaded_mps= json.loads(mps_content)

#normalize it (to flatten out the hierarchy) and create a pandas dataframe to Do Stuff With
mps = pd.json_normalize(loaded_mps)

mps.head()



api status:  200


,id,latest_member.id,latest_member.name.first,latest_member.name.last,latest_member.electorate,latest_member.house,latest_member.party
0,10007,6,Anthony,Albanese,Grayndler,representatives,Australian Labor Party
1,10014,13,Kevin,Andrews,Menzies,representatives,Liberal Party
2,10043,40,Sharon,Bird,Cunningham,representatives,Australian Labor Party
3,10081,69,Tony,Burke,Watson,representatives,Australian Labor Party
4,10088,74,Anthony,Byrne,Holt,representatives,Australian Labor Party


In [4]:
#This is writing & calling a function to view the JSON directly (to help clarify what the structure is prior to normalizing it)
#def jprint(obj):
    # create a formatted string of the Python JSON object
#   text = json.dumps(obj, sort_keys=True, indent=4)
#   print(text)
    
#jprint(mps_get.json())



In [22]:
#Exploring the dataset


#rename cols for readability
mps = mps.rename(columns={'latest_member.id':'member_id'})
mps.columns = mps.columns.str.replace('latest_member.','')
mps = mps.rename(columns={'name.first':'first_name','name.last':'last_name'})

print(mps.columns)

print(mps.groupby(['house','party']).size())
print(mps['party'].value_counts())

Index(['id', 'member_id', 'first_name', 'last_name', 'electorate', 'house',
       'party'],
      dtype='object')
house            party                            
representatives  Australian Greens                     1
                 Australian Labor Party               68
                 CWM                                   1
                 Centre Alliance                       1
                 Independent                           4
                 Katter's Australian Party             1
                 Liberal National Party                5
                 Liberal Party                        54
                 National Party                       15
                 SPK                                   1
senate           Australian Greens                     9
                 Australian Labor Party               25
                 Centre Alliance                       1
                 Country Liberal Party                 1
                 DPRES              

In [24]:
#Export the MPs to CSV

mps.to_csv("..\Data\mps.csv", index=False)

# Now we will switch to looking at some policies.

In [17]:
#Api call
policies_get = requests.get("https://theyvoteforyou.org.au/api/v1/policies.json?key=HsjbG%2FoZdGMFMbN4PK%2F1")

#Print the api status to confirm the connection etc
print("api status: ", policies_get.status_code)

#pull the content from the api
policies_content = policies_get.content

#Load the content into a Python object
policies_loaded = json.loads(policies_content)

#normalize it (to flatten out the hierarchy) and create a pandas dataframe to Do Stuff With
policies = pd.json_normalize(policies_loaded)

policies

api status:  200


,id,name,description,provisional
0,1,same-sex marriage equality,the Federal Government should amend the Marria...,False
1,2,tobacco plain packaging,the federal government should introduce plain ...,False
2,3,a carbon price,the federal government should introduce a carb...,False
3,4,increasing scrutiny of asylum seeker management,there should be more independent access to det...,False
4,5,government administered paid parental leave,the federal government should administer the p...,False


In [25]:
#Export the policies to CSV

policies.to_csv("..\Data\policies.csv", index=False)